# 1. Import and Install Dependencies

In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [ ]:
import cv2
import numpy as np
import os


In [ ]:
import pyzed.sl as sl

# 2. Setup Folders for Collection

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['boxing', 'notFighting', 'kick'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [ ]:
# create the folder DATA_PATH if it does not exist and make all the actions as subfolders
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [ ]:
for action in actions: 
    dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
        except:
            pass

# 3. Setup ZED2i

In [ ]:
def draw_skeleton(frame, keypoint_2d):
    # Define connections between keypoints if not , e.g., [(0, 1), (1, 2), ...]
    # Defines keypoints for Zed2i SDK
    skeleton_pairs = [
        # Spine
        (0, 1), (1, 2), (2, 3), (3, 4),
        # Head
        (4, 5), (5, 6), (5, 7), (6, 8), (7, 9),
        # Arms
        (4, 10), (10, 12), (12, 14), (14, 16),  # Left arm
        (4, 11), (11, 13), (13, 15), (15, 17),  # Right arm
        # Legs
        (0, 18), (18, 20), (20, 22), (22, 24), (22, 26), (22, 28),  # Left leg
        (0, 19), (19, 21), (21, 23), (23, 25), (23, 27), (23, 29),  # Right leg
        # Fingers (only showing the first and last joint for brevity)
        (16, 30), (16, 32), (16, 34), (16, 36),  # Left hand fingers
        (17, 31), (17, 33), (17, 35), (17, 37),  # Right hand fingers
    ]


    for start, end in skeleton_pairs:
        start_point = tuple(map(int, keypoint_2d[start]))
        end_point = tuple(map(int, keypoint_2d[end]))

        # Draw lines for bones
        cv2.line(frame, start_point, end_point, (0, 255, 0), 2)

        # Draw circles for joints
        cv2.circle(frame, start_point, 3, (0, 0, 255), -1)
        cv2.circle(frame, end_point, 3, (0, 0, 255), -1)

# 4. Collect Keypoint Values for Training and Testing

In [ ]:
import csv
def write_keypoints_to_csv(action_class, keypoints, filename):
    #Create a header for the CSV file
    header = ['class']
    for val in range(1, len(keypoints) + 1):
        header.append(f"x{val}")
        header.append(f"y{val}")

    # Flatten the keypoints list
    flat_keypoints = [coord for kp in keypoints for coord in kp]

    print(flat_keypoints)
    # Open the CSV file for writing and appending
    with open(filename, mode='a', newline='') as file:
        csv_writer = csv.writer(file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
        # Write the header only once at the beginning
        if file.tell() == 0:
            csv_writer.writerow(header)
        
        # Write the keypoints, prefixed with the action class
        row = [action_class] + flat_keypoints
        csv_writer.writerow(row)


def write_keypoints_to_csv3D(action_class, keypoints, filename):
    #Create a header for the CSV file
    header = ['class']
    for val in range(1, len(keypoints) + 1):
        header.append(f"x{val}")
        header.append(f"y{val}")
        header.append(f"z{val}")

    # Flatten the keypoints list
    flat_keypoints = [coord for kp in keypoints for coord in kp]

    print(flat_keypoints)
    # Open the CSV file for writing and appending
    with open(filename, mode='a', newline='') as file:
        csv_writer = csv.writer(file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
        # Write the header only once at the beginning
        if file.tell() == 0:
            csv_writer.writerow(header)
        
        # Write the keypoints, prefixed with the action class
        row = [action_class] + flat_keypoints
        csv_writer.writerow(row)



In [ ]:

actions = np.array(['boxing', 'notFighting', 'kick'])
no_sequences = 30
sequence_length = 30
DATA_PATH = os.path.join('MP_Data') 


for actionses in actions:
    for sequence in range(0, no_sequences + 1):
        try:
            os.makedirs(os.path.join(DATA_PATH, actionses, str(sequence)))
        except:
            pass


In [ ]:
 # Create a Camera object
zed = sl.Camera()

# Create a InitParameters object and set configuration parameters
init_params = sl.InitParameters()
#init_params.camera_resolution = sl.RESOLUTION.HD720  # Use HD720 video mode
init_params.depth_mode = sl.DEPTH_MODE.PERFORMANCE
init_params.coordinate_units = sl.UNIT.METER
init_params.sdk_verbose = 1

init_params.camera_resolution = sl.RESOLUTION.HD1080
init_params.camera_fps = 30



# Open the camera
err = zed.open(init_params)
if err != sl.ERROR_CODE.SUCCESS:
    print("Camera Open : "+repr(err)+". Exit program.")
    exit()

body_params = sl.BodyTrackingParameters()
# Different model can be chosen, optimizing the runtime or the accuracy
body_params.detection_model = sl.BODY_TRACKING_MODEL.HUMAN_BODY_FAST
body_params.enable_tracking = True
body_params.image_sync = True
body_params.enable_segmentation = False
# Optimize the person joints position, requires more computations
body_params.enable_body_fitting = True
body_params.body_format = sl.BODY_FORMAT.BODY_38

if body_params.enable_tracking:
    positional_tracking_param = sl.PositionalTrackingParameters()
    # positional_tracking_param.set_as_static = True
    positional_tracking_param.set_floor_as_origin = True
    zed.enable_positional_tracking(positional_tracking_param)

print("Body tracking: Loading Module...")

err = zed.enable_body_tracking(body_params)
if err != sl.ERROR_CODE.SUCCESS:
    print("Enable Body Tracking : "+repr(err)+". Exit program.")
    zed.close()
    exit()
bodies = sl.Bodies()
body_runtime_param = sl.BodyTrackingRuntimeParameters()
# For outdoor scene or long range, the confidence should be lowered to avoid missing detections (~20-30)
# For indoor scene or closer range, a higher confidence limits the risk of false positives and increase the precision (~50+)
body_runtime_param.detection_confidence_threshold = 40


cv2.namedWindow("ZED Body Tracking", cv2.WINDOW_NORMAL)
image = sl.Mat()
i = 0 

cap = cv2.VideoCapture(0)
# NEW LOOP
# Loop through actions
for action in actions:
    # Loop through sequences aka videos
    for sequence in range(no_sequences):
        # Loop through video length aka sequence length
        for frame_num in range(sequence_length):
            
            if sequence == 0:
                #go to next sequence
                continue
            if zed.grab() == sl.ERROR_CODE.SUCCESS:
                
                zed.retrieve_image(image, sl.VIEW.LEFT)  # Get the image
                frame = image.get_data()  # Convert to OpenCV format
                err = zed.retrieve_bodies(bodies, body_runtime_param)
                if bodies.is_new:
                    body_array = bodies.body_list

                    if len(body_array) > 0:

                        #make a 30 seconds counter before starting to record
                        for body in body_array:

                            body = body_array[0]
                            keypoint_2d = body.keypoint_2d
                            keypoint_3d = body.keypoint
                            draw_skeleton(frame, keypoint_2d)
                            print(keypoint_2d[15])
                            cv2.imshow('OpenCV Feed', frame)

                            # NEW Apply wait logic
                            if frame_num == 0: 
                                cv2.putText(frame, 'STARTING COLLECTION', (120,200), 
                                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                                cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                                # Show to screen
                                cv2.imshow('OpenCV Feed', frame)
                                cv2.waitKey(5000)
                            else: 
                                cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                                # Show to screen
                                cv2.imshow('OpenCV Feed', frame)
                             
                             # NEW Export keypoints
                            write_keypoints_to_csv3D(action, keypoint_2d, 'keypoints.csv')
                            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                            np.save(npy_path, keypoint_2d)

                            # Break gracefully
                            if cv2.waitKey(10) & 0xFF == ord('q'):
                                break
                    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
#Create a header for the CSV file
header = ['class']
for val in range(1, len(keypoint_3d) + 1):
    print(val)

# 5. Preprocess Data and Create Labels and Features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        if sequence == 0:
            continue
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(labels).shape

In [ ]:
X = np.array(sequences)

In [ ]:
X = X.reshape((X.shape[0], X.shape[1], -1))


In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Reshape data for scaling
nsamples, nx, ny = X_train.shape
X_train_reshaped = X_train.reshape((nsamples*nx, ny))

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reshaped)

# Reshape data back to original shape
X_train_scaled = X_train_scaled.reshape((nsamples, nx, ny))

# 6. Build and Train LSTM Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.regularizers import l2


In [ ]:
log_dir = os.path.join("Logs")
tb_callback= TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(32, return_sequences=False, input_shape=(30,76), activation='tanh',
               kernel_regularizer=l2(0.01)))  # L2 regularization
model.add(Dropout(0.5))  # Dropout
model.add(Dense(16, activation="relu"))
#model.add(Dropout(0.5))  # Dropout
model.add(Dense(actions.shape[0], activation="softmax"))


In [ ]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.00007, clipnorm=1.0)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=100, mode='min', verbose=1)

model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback, early_stopping], validation_split=0.1)


In [ ]:
model.summary()

# Make a prediction

In [ ]:
model.predict(X_test)

In [ ]:
model.predict(X_test)[2]

In [ ]:
actions[np.argmax(model.predict(X_test)[3])]



In [ ]:
actions[np.argmax(y_test[3])]

In [ ]:
model.save('mainModel.h5')
del model

# 7. Evaluate

In [ ]:
model.load_weights('mainModel.h5')

In [ ]:
lstm_layer = model.layers[0]

# Extract all weights from the LSTM layer
weights = lstm_layer.get_weights()  # Returns a list of numpy arrays

# LSTM weights include input, recurrent (hidden), and biases
# weights[0] are input weights, [1] are recurrent weights, [2] are biases

# Input weights to the LSTM for skeleton point k
input_weights_k = weights[0][:, 5]

# Output or analyze the extracted weights
print("Input weights for skeleton point k:", input_weights_k)

In [ ]:
def compute_accuracy(predictions, labels):
    # Assuming both inputs are numpy arrays and labels are one-hot encoded
    predicted_classes = np.argmax(predictions, axis=-1)
    true_classes = np.argmax(labels, axis=-1)
    accuracy = np.mean(predicted_classes == true_classes)
    return accuracy

# Predicting on original data
original_predictions = model.predict(X_test)
original_accuracy = compute_accuracy(original_predictions, y_test)
print(original_accuracy)

num_features = X_test.shape[2]  # Number of skeleton points
importances = np.zeros(num_features)

### Normalized Keypoint Importance for Actions


In [ ]:
from sklearn.metrics import accuracy_score


# Filter out the samples for the 'boxing' action
boxing_index = np.where(np.argmax(y, axis=1) == label_map['boxing'])[0]
X_boxing = X[boxing_index]
y_boxing = y[boxing_index]


# Predict with the original 'boxing' test set
original_predictions = model.predict(X_boxing)
original_pred_labels = np.argmax(original_predictions, axis=1)
y_boxing_labels = np.argmax(y_boxing, axis=1)
original_accuracy = accuracy_score(y_boxing_labels, original_pred_labels)

# Store the accuracy change for 'boxing'
accuracy_changes_boxing = []

# Perform feature ablation for 'boxing'
for i in range(76):  # Loop over each feature
    X_ablated = X_boxing.copy()
    X_ablated[:, :, i] = 0  # Zero out the feature
    ablated_predictions = model.predict(X_ablated)
    ablated_pred_labels = np.argmax(ablated_predictions, axis=1)
    change_in_accuracy = original_accuracy - accuracy_score(y_boxing_labels, ablated_pred_labels)
    accuracy_changes_boxing.append(change_in_accuracy)

# Calculate and normalize the importance for 'boxing'
keypoint_importance_boxing = np.zeros(38)
for i in range(0, 76, 2):
    keypoint_index = i // 2
    keypoint_importance_boxing[keypoint_index] = accuracy_changes_boxing[i] + accuracy_changes_boxing[i + 1]
keypoint_importance_boxing = np.abs(keypoint_importance_boxing)
normalized_keypoint_importance_boxing = keypoint_importance_boxing / np.sum(keypoint_importance_boxing)

# Plot the importance for 'boxing'
plt.figure(figsize=(15, 5))
plt.bar(range(38), normalized_keypoint_importance_boxing)
plt.title('Normalized Keypoint Importance for boxing Action')
plt.xlabel('Keypoint Index')
plt.ylabel('Normalized Importance')
plt.xticks(range(38))  # Add keypoint names if available
plt.show()


### Normalized Keypoint Importance for model based on testset

In [ ]:
# Predict with the original test set
original_predictions = model.predict(X_test)
# Convert one-hot encoded predictions to label indices
original_pred_labels = np.argmax(original_predictions, axis=1)
# Convert one-hot encoded true labels to label indices
y_test_labels = np.argmax(y_test, axis=1)
# Calculate the original accuracy
original_accuracy = accuracy_score(y_test_labels, original_pred_labels)

# Store the accuracy change
accuracy_changes = []

# Perform feature ablation
for i in range(76):  # Loop over each feature (38 keypoints * 2 for x and y)
    # Copy X_test to not alter original data
    X_ablated = X_test.copy()
    
    # Zero out the feature across all samples and time steps
    X_ablated[:, :, i] = 0
    
    # Predict with the ablated test set
    ablated_predictions = model.predict(X_ablated)
    # Convert one-hot encoded predictions to label indices
    ablated_pred_labels = np.argmax(ablated_predictions, axis=1)
    
    # Calculate the change in accuracy
    change_in_accuracy = original_accuracy - accuracy_score(y_test_labels, ablated_pred_labels)
    accuracy_changes.append(change_in_accuracy)

# Initialize an array for summed importance of keypoints
keypoint_importance = np.zeros(38)

# Sum the importance of x and y for each keypoint
for i in range(0, 76, 2):  # Increment by 2 since we are processing pairs (x, y)
    keypoint_index = i // 2  # Integer division to get keypoint index
    keypoint_importance[keypoint_index] = accuracy_changes[i] + accuracy_changes[i + 1]

# Normalize the importance values
normalized_keypoint_importance = keypoint_importance / np.sum(keypoint_importance)

# Plot the importance of keypoints
plt.figure(figsize=(15, 5))
plt.bar(range(len(normalized_keypoint_importance)), normalized_keypoint_importance)
plt.title('Normalized Importance of Each Skeleton Key Point')
plt.xlabel('Skeleton Key Point Index')
plt.ylabel('Normalized Decrease in Accuracy')
plt.xticks(ticks=range(len(keypoint_names)), labels=keypoint_names, rotation=90)
plt.show()


### Normalized Keypoint Importance for all of models keypoints

In [ ]:
# Get the input weights to the LSTM layer (76, 128)
lstm_layer = model.layers[0]
weights = lstm_layer.get_weights()[0]

# Sum the absolute values of weights for each of the 76 input features
feature_importance = np.sum(np.abs(weights), axis=1)

# Since each keypoint has an x and y coordinate, sum the importances for the x and y of each keypoint
keypoint_importances = np.array([np.sum(feature_importance[i*2:i*2+2]) for i in range(38)])

# Normalize the importances to make them relative
normalized_importances = keypoint_importances / np.sum(keypoint_importances)

# Plotting
plt.figure(figsize=(20, 8))
keypoint_names = [f'{i}' for i in range(38)]  # Replace with actual keypoint names if available
plt.bar(keypoint_names, normalized_importances)
plt.title('Normalized Importance of Each Skeleton Key Point')
plt.xlabel('Skeleton Key Points')
plt.ylabel('Normalized Importance')
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout so everything fits without overlapping
plt.show()

In [ ]:
# Example of manually modifying a single feature across all samples
test_input = copy.deepcopy(X_test)
test_input[:, :, 0] += 1.0  # Add a constant value to the first feature across all samples
altered_predictions = model.predict(test_input)
altered_accuracy = compute_accuracy(altered_predictions, y_test)
print("Original Accuracy:", original_accuracy)
print("Altered Accuracy:", altered_accuracy)


### Framework for generating conf matrix,accuracy score and f1 score of an observed test of the system

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, multilabel_confusion_matrix
import seaborn as sns

# TEST2.mp4
predictions = [
    0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2,
    1, 2, 2, 2, 1, 2, 1, 2, 0, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2,
    1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2,
    1, 2, 1, 2, 0, 2, 0, 2, 0, 2, 0, 2, 1, 2, 1, 2, 0, 2, 1, 2
]
true_labels = [
    0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2,
    1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2,
    1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2,
    1, 2, 1, 2, 0, 2, 0, 2, 0, 2, 0, 2, 1, 2, 1, 2, 0, 2, 1, 2
]

# Ensure the number of values is 80
assert len(predictions) == 80
assert len(true_labels) == 80

# Split the labels and predictions into three groups
true_labels_1 = true_labels[:20]  # First 20 for Boxing and Nonfighting
predictions_1 = predictions[:20]

true_labels_2 = true_labels[20:40]  # Next 20 for Kick and Nonfighting
predictions_2 = predictions[20:40]

true_labels_3 = true_labels[40:]  # Last 40 for mixed (Boxing to Kick)
predictions_3 = predictions[40:]

# Compute confusion matrices
cm1 = confusion_matrix(true_labels_1, predictions_1, labels=[0, 1, 2])
cm2 = confusion_matrix(true_labels_2, predictions_2, labels=[0, 1, 2])
cm3 = confusion_matrix(true_labels_3, predictions_3, labels=[0, 1, 2])

# Compute performance metrics
def compute_metrics(cm):
    true_labels = []
    predicted_labels = []
    for i in range(len(cm)):
        for j in range(len(cm[i])):
            true_labels.extend([i] * cm[i][j])
            predicted_labels.extend([j] * cm[i][j])
    accuracy = accuracy_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    return accuracy, f1

accuracy1, f1_1 = compute_metrics(cm1)
accuracy2, f1_2 = compute_metrics(cm2)
accuracy3, f1_3 = compute_metrics(cm3)

avg_accuracy = np.mean([accuracy1, accuracy2, accuracy3])
avg_f1_score = np.mean([f1_1, f1_2, f1_3])

# Print the averaged performance metrics
print(f"Average Accuracy: {avg_accuracy:.2f}")
print(f"Average F1 Score: {avg_f1_score:.2f}")

# Plot confusion matrices
def plot_confusion_matrix(cm, title):
    actions = ['Boxing', 'Kick', 'Nonfighting']
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="viridis", xticklabels=actions, yticklabels=actions, annot_kws={"size": 16})
    plt.xlabel('Predicted label', fontsize=14)
    plt.ylabel('True label', fontsize=14)
    plt.title(title, fontsize=16)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()

plot_confusion_matrix(cm1, 'Confusion Matrix 1 (Boxing and Nonfighting)')
plot_confusion_matrix(cm2, 'Confusion Matrix 2 (Kick and Nonfighting)')
plot_confusion_matrix(cm3, 'Confusion Matrix 3 (Mixed Boxing and Kick)')

# Generate the multilabel confusion matrix
conf_matrix = multilabel_confusion_matrix(true_labels, predictions)
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')

print("Overall Accuracy: {:.2f}".format(accuracy))
print("Overall F1 Score: {:.2f}".format(f1))
print(conf_matrix)

# Plot the multilabel confusion matrices
def plot_multilabel_confusion_matrix(conf_matrix):
    labels = ['Boxing', 'Kick', 'Nonfighting']
    for i, cm in enumerate(conf_matrix):
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt="d", cmap="viridis", xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'], annot_kws={"size": 16})
        plt.xlabel('Predicted', fontsize=14)
        plt.ylabel('True', fontsize=14)
        plt.title(f'Confusion Matrix for {labels[i]}', fontsize=16)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.show()

plot_multilabel_confusion_matrix(conf_matrix)


In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1)

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
y_true = true_labels  
y_pred = predictions  

f1 = f1_score(y_true, y_pred, average='weighted')
print(f1)


# 8. Test in real time

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        if num < len(colors):  # Check if the index is within the range of colors
            cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
            cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(cv2.cvtColor(prob_viz(model.predict(X_test)[0], actions, np.zeros((100,100,3), dtype=np.uint8), colors), cv2.COLOR_BGR2RGB))

In [ ]:
 # Create a Camera object
zed = sl.Camera()

# Create a InitParameters object and set configuration parameters
init_params = sl.InitParameters()
#init_params.camera_resolution = sl.RESOLUTION.HD720  # Use HD720 video mode
init_params.depth_mode = sl.DEPTH_MODE.PERFORMANCE
init_params.coordinate_units = sl.UNIT.METER
init_params.sdk_verbose = 1

init_params.camera_resolution = sl.RESOLUTION.HD1080
init_params.camera_fps = 30


# Open the camera
err = zed.open(init_params)
if err != sl.ERROR_CODE.SUCCESS:
    print("Camera Open : "+repr(err)+". Exit program.")
    exit()

body_params = sl.BodyTrackingParameters()
# Different model can be chosen, optimizing the runtime or the accuracy
body_params.detection_model = sl.BODY_TRACKING_MODEL.HUMAN_BODY_FAST
body_params.enable_tracking = True
body_params.image_sync = True
body_params.enable_segmentation = False
# Optimize the person joints position, requires more computations
body_params.enable_body_fitting = True
body_params.body_format = sl.BODY_FORMAT.BODY_38

if body_params.enable_tracking:
    positional_tracking_param = sl.PositionalTrackingParameters()
    # positional_tracking_param.set_as_static = True
    positional_tracking_param.set_floor_as_origin = True
    zed.enable_positional_tracking(positional_tracking_param)

print("Body tracking: Loading Module...")

err = zed.enable_body_tracking(body_params)
if err != sl.ERROR_CODE.SUCCESS:
    print("Enable Body Tracking : "+repr(err)+". Exit program.")
    zed.close()
    exit()
bodies = sl.Bodies()
body_runtime_param = sl.BodyTrackingRuntimeParameters()
# For outdoor scene or long range, the confidence should be lowered to avoid missing detections (~20-30)
# For indoor scene or closer range, a higher confidence limits the risk of false positives and increase the precision (~50+)
body_runtime_param.detection_confidence_threshold = 40


cv2.namedWindow("ZED Body Tracking", cv2.WINDOW_NORMAL)
image = sl.Mat()
i = 0 

cap = cv2.VideoCapture(0)
# NEW LOOP
# Loop through actions

sequencelist = []
sentence = []
threshold = 0.5



while True:
        if zed.grab() == sl.ERROR_CODE.SUCCESS:
            zed.retrieve_image(image, sl.VIEW.LEFT)  # Get the image
            frame = image.get_data()  # Convert to OpenCV format
            err = zed.retrieve_bodies(bodies, body_runtime_param)
            if bodies.is_new:
                body_array = bodies.body_list
                
                if len(body_array) > 0:


                    body = body_array[0]

                        
                        # Extract 2D keypoints
                    keypoint_2d = body.keypoint_2d
                    print("Keypoints 2D: " + str(keypoint_2d))
                    
                    


                    # Draw skeleton joints on the image
                    draw_skeleton(frame, keypoint_2d)


                    frame_copy = frame.copy()




                    # predict logic
                    processed_data = np.array(keypoint_2d).reshape(1, -1)

                    sequencelist.append(processed_data)

                    sequencelist = sequencelist[-30:]


                    sequencelista = np.stack(sequencelist, axis=0)
                    sequencelista = np.squeeze(sequencelist, axis=1)
                    
                    if len(sequencelist) == 30:
                        v = np.expand_dims(sequencelista, axis=0)
                        res = model.predict(v)[0]
                        print(actions[np.argmax(res)])


                        if res[np.argmax(res)] > threshold:
                            if len(sentence) > 0:
                                if actions[np.argmax(res)] != sentence[-1]:
                                    sentence.append(actions[np.argmax(res)])
                            else:
                                sentence.append(actions[np.argmax(res)])


                        
                        if len(sentence) > 1:
                            sentence = sentence[-1:]
                        # Show the sentence text

                        frame_copy = prob_viz(res, actions, frame_copy, colors)
                        
                        sentence_text = ' '.join(sentence)
                        cv2.putText(frame_copy,sentence_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                        
                        cv2.rectangle(frame_copy, (0, 0), (640, 40), (0, 255, 0), 2)

                    
                        # Show to screen
                        cv2.imshow('OpenCV Feed', frame_copy)

                    
            #cv2.imshow("ZED Body Tracking", frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
# Close the camera
zed.disable_body_tracking()
zed.close()
cv2.destroyAllWindows()

# 9. Generate .onnx file for model

In [ ]:
!pip install tf2onnx


In [ ]:
from tensorflow.keras.models import load_model

# Load the Keras model from the HDF5 file
model = load_model('mainModel.h5')

import tf2onnx
import tensorflow as tf

# The input name and shape can vary depending on your model's architecture
# For example, if your model's input layer name is 'input_1' and expects input of size (224, 224, 3), specify that here
input_spec = [tf.TensorSpec((None, 30, 76), tf.float32, name="input")]

# Convert the model
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=input_spec, opset=13)

# Save the ONNX model to a file
onnx_file_name = "modelfunkar.onnx"
with open(onnx_file_name, "wb") as f:
    f.write(onnx_model.SerializeToString())

